<img src="./University_Debrecen_logo.jpg" alt="Drawing" style="width: 200px;"/>

# Processing the recordings containing SIMV-VG-PS ventilation mode

### 1. Import the required modules

In [ ]:
import pickle
import sys
import copy
import os
import gc
import copy
import warnings
from collections import defaultdict
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pandas import DataFrame, Series

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

pd.options.mode.chained_assignment = None

In [ ]:
print(f'Python version: {sys.version}')
print(f'pandas version: {pd.__version__}')
print(f'matplotlib version: {mpl.__version__}')
print(f'numPy version: {np.__version__}')

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'SIMV_VG_PS'

# Directory containing clinical and blood gas data
DIR_READ_CLIN = os.path.join(os.sep, 'Users', 'guszti', 'ventilation_draeger_debrecen')

# Folder on external drive to read the ventilation data from
DIR_READ_VENT =  os.path.join(os.sep, 'Volumes', 'Guszti', 'draeger_debrecen',)

# Folder to write statistics and reports on the group
DIR_WRITE = os.path.join(os.sep, 'Users', 'guszti', 'ventilation_draeger_debrecen', 'Analyses', TOPIC)
os.makedirs(DIR_WRITE, exist_ok = True)

# Folder on external drive to export graphs and data about individual recordings
DATA_DUMP = os.path.join(os.sep, 'Volumes', 'Guszti', 'data_dump', 'draeger_debrecen', TOPIC)
os.makedirs(DATA_DUMP, exist_ok = True)

In [ ]:
DIR_READ_CLIN, DIR_READ_VENT, DIR_WRITE, DATA_DUMP

### 3. Import processed clinical details

This recording list is produced by the `Clinical_details_processing_debrecen.ipynb` notebook

In [ ]:
clinical_details_recordings = pd.read_csv(os.path.join(DIR_READ_CLIN, 'clinical_details_recordings.csv'))

In [ ]:
clinical_details_recordings.head()

In [ ]:
clinical_details_patients = pd.read_csv(os.path.join(DIR_READ_CLIN, 'clinical_details_patients.csv'))

In [ ]:
clinical_details_patients

In [ ]:
# Generate a list of patients
patients = list(clinical_details_patients['Patient'])
# Limit the study for the first 20 patients
patients = [patient for patient in patients if int(patient[-2:]) <= 20]
print(patients)

### 4. Import ventilator modes

For some recordings of _LVD002_, the ventilator modes (slowText) file is empty. For now I am assuming that they were all SIMV-VG with PS. This needs to be verified later.

In [ ]:
%%time

vent_modes = defaultdict(dict)

for patient in patients:
    
    # Create nested dictionary for each recording
    vent_modes_files = [fle for fle in os.listdir(os.path.join(DIR_READ_VENT, patient)) if '_slow_Text' in fle]

    for fle in vent_modes_files:
        
        try: # Some of the slow Text files are empty
        
            path = os.path.join(DIR_READ_VENT, patient, fle,)
            # Use the specific part of the filename as a unique key for the internal dictionary
            tag = fle[11:32]
            # Import data, parse the 'Date' and 'Time' columns as datetime and combine them 
            vent_modes[patient][tag] = pd.read_csv(path, parse_dates = [['Date', 'Time']])
            # Set the combined 'Date_Time' column as row index
            vent_modes[patient][tag] = vent_modes[patient][tag].set_index('Date_Time')
            # Drop irrelevant column
            vent_modes[patient][tag].drop('Rel.Time [s]', axis = 1, inplace = True)
            
        except Exception as e:
            print(patient, tag, e)
            vent_modes[patient][tag] = DataFrame()

In [ ]:
vent_modes.keys()

In [ ]:
vent_modes['LVD006']['2021-11-30_142736.417']

### 5. Import ventilator settings

In [ ]:
def file_finder(recording, tag):
    # There are sometime hidden files on the hard drive starting with '.' 
    # this step is necessary to ignore them
    flist = sorted(file for file in os.listdir(os.path.join(DIR_READ_VENT, patient)) if not file.startswith('.'))
    
    #Takes a list of filenames and returns those ones that which contain 'tag'
    return [os.path.join(DIR_READ_VENT, patient, fname) for fname in flist if tag in fname]

In [ ]:
def data_loader(path):
    # This is escaping characters with encoding errors with blackslashes that pd.csv can subsequently handle
    with open(path, encoding='utf8' , errors = 'backslashreplace') as input_fd:
        data = pd.read_csv(input_fd, keep_date_col = 'True', parse_dates = [['Date', 'Time']])
    data.index = data['Date_Time']
    return data

In [ ]:
def process_files(patient, tag):
    data = {}
    paths = file_finder(patient, tag)
    for path in paths:
        data['_'.join(path.split('_')[-4:-2])] = data_loader(path)
    return data

In [ ]:
def col_renamer(col_name):

    # `VTi` is not correctly labelled, it is actually the target (leak-compensated) expired tidal volume
    old_names = ['FiO2', 'VTi', 'Slope', 'Ti', 'Te', 'RR', 
                 'Pinsp', 'PEEP', 'Pmax', 'Flow trigger', 'ΔPsupp', 'Timax',
                 'MAPhf', 'VThf', 'Ampl hf', 'Ampl hf max', 'fhf', 'VG']
    new_names = ['FiO2_set [%]', 'VT_set [mL]', 'Slope_set [s]', 'Ti_set [s]', 'Te_set [s]', 'RR_set [1/min]', 
                 'PIP_set [mbar]', 'PEEP_set [mbar]', 'Pmax_set [mbar]', 'Flow_trigger_set [L/min]', 'PS_set [mbar]', 'Timax_set [s]', 
                 'MAPhf_set [mbar]', 'VThf_set [mL]', 'Amplhf_set [mbar]', 'Amplhfmax_set [mbar]', 'fhf_set [Hz]', 'VG']
    rename_dict = dict(zip(old_names, new_names))
        
    return rename_dict[col_name]

In [ ]:
def settings_cleaner(dct):
    settings_to_keep = ['FiO2', 'VTi', 'VT', 'Slope', 'Ti', 'Te', 'RR', 'Pinsp', 'PEEP', 'Pmax', 
        'Flow trigger', 'ΔPsupp','Timax','MAPhf', 'VThf', 'Ampl hf', 'Ampl hf max', 'fhf', ]
    
    for part in dct:
        dframe = dct[part]
        # This is required as VTi is given both in mL and in L and keeping both would 
        # create duplicate columns during unstacking leading to an error message
        dframe = dframe[dframe['Unit'] != 'L'] 
        dframe = dframe[dframe['Id'].isin(settings_to_keep)] 
        
        # Create a new DataFrame with info if the VG was on or not
        VG_dframe = pd.DataFrame(index = dframe.index.copy())
        VG_dframe['VG'] = np.nan 
        if 'VTi' in dframe['Id'].unique() and 'Pinsp' in dframe['Id'].unique(): 
            VG_dframe['VG'][dframe['Id'] == 'VTi'] = 'on' # VG is on
            VG_dframe['VG'][dframe['Id'] == 'Pinsp'] = 'off' # VG is off
        
        elif 'VTi' in dframe['Id'].unique() and 'Pinsp' not in dframe['Id'].unique():
            VG_dframe['VG'] = 'on' # VG is on
        
        elif 'VTi' not in dframe['Id'].unique() and 'Pinsp' in dframe['Id'].unique():
            VG_dframe['VG'] = 'off' # VG is off
        
        VG_dframe['VG'] = VG_dframe['VG'].astype('category')
        dframe = dframe.pivot('Date_Time', 'Id', 'Value New')
        dframe = pd.merge(dframe, VG_dframe, how = 'outer', left_index = True, right_index = True)
        dframe = dframe.fillna(method = 'ffill')
        # Rows with duplicated indices need to be removed as otherwise DataFrame cannot be merged with
        # slow_measurements. All these rows contain duplicated data anyway.
        dframe = dframe[~dframe.index.duplicated(keep='last')]
        dframe.rename(col_renamer, axis = 1, inplace = True)
        # Remove the microseconds from the time stamp
        dframe.index = dframe.index.values.astype('datetime64[s]')
        
        dct[part] = dframe
    
    return dct

In [ ]:
%%time

vent_settings = {}
for patient in patients:
    vent_settings[patient] = process_files(patient, 'slow_Setting')
    vent_settings[patient] = settings_cleaner(vent_settings[patient])

In [ ]:
# Check which ventilator settings are present in any recording
parameter_present_in_any = set()
for patient in vent_settings:
    for recording in vent_settings[patient]:
        parameter_present_in_any.update(vent_settings[patient][recording].columns)

parameter_present_in_any

In [ ]:
# Check which ventilator settings are present in all recordings
parameter_present_in_all = parameter_present_in_any.copy()
for patient in vent_settings:
    for recording in vent_settings[patient]:
        parameter_present_in_all &= set(vent_settings[patient][recording].columns)

parameter_present_in_all

In some recordings Timax has been set although there was no PSV mode used. Please clarify this.

In some recordings PIP has been also set suggesting that at least part of the recording was with VG off. This info will be inserted in the DataFrame.

In [ ]:
# Check which ventilator settings are present extra in  each recording
for patient in vent_settings:
    for recording in vent_settings[patient]:
        extra_pars = set(vent_settings[patient][recording].columns) - parameter_present_in_all
        if extra_pars:
            print(patient, recording, extra_pars)

In [ ]:
vent_modes['LVD006']['2021-11-30_142736.417'];

In [ ]:
vent_settings['LVD006']['2021-11-30_142736.417'];

In [ ]:
vent_modes['LVD008']['2021-12-04_132746.283'];

In [ ]:
vent_settings['LVD008']['2021-12-04_132746.283'];

In [ ]:
# This is an example when the recording has both VG and noVG
vent_settings['LVD002']['2021-11-08_105631.989'];

In [ ]:
# Check if there are duplicated columns:
for patient in vent_settings:
    for recording in vent_settings[patient]:
        if vent_settings[patient][recording].columns.duplicated().any():
                print(f'Some columns are duplicated for {patient} {recording}')
        else:
            print(f'No duplicated columns for {patient} {recording}')

### 6. Identify which recordings had only SIMV with or without VG

In [ ]:
has_cmv = defaultdict(int); has_sippv = defaultdict(int); has_simv = defaultdict(int); 
has_psv = defaultdict(int); has_mmv = defaultdict(int); has_hfov = defaultdict(int); 
has_vg = defaultdict(int); has_novg = defaultdict(int)

for patient in patients:
    for recording in vent_modes[patient]:
        try:
            entries = [value.strip(' ') for value in vent_modes[patient][recording]['Text'].values]
        
            if 'Mode PC-CMV' in entries:
                has_cmv[patient, recording] = 1
        
            if 'Mode PC-AC' in entries:
                has_sippv[patient, recording] = 1

            if 'Mode PC-SIMV' in entries:
                has_simv[patient, recording] = 1
        
            if 'Mode PC-PSV' in entries:
                has_psv[patient, recording] = 1
       
            if 'Mode PC-MMV' in entries:
                has_mmv[patient, recording] = 1
       
            if 'Mode PC-HFO' in entries:
                has_hfov[patient, recording] = 1
       
            if '/VG' in entries:
                has_vg[patient, recording] = 1
                
        except Exception as e:
            # Some recordings have no slow_Text files but mode was always SIMV
            print(f'No data stored for {patient}, {recording}', e)
            has_simv[patient, recording] = 1
            continue
            
        # This is to identify recordings when VG was off during (or during part of) the recording
        if 'PIP_set [mbar]' in vent_settings[patient][recording].columns:
            has_novg[patient, recording] = 1  
            
    
has_vent_mode = DataFrame([has_cmv, has_sippv, has_simv, has_psv, has_mmv, has_hfov, has_vg, has_novg]).T
has_vent_mode.columns = ['cmv', 'sippv', 'simv', 'psv', 'mmv', 'hfov', 'vg', 'novg']
has_vent_mode.fillna(0, inplace = True)
has_vent_mode.sort_index(inplace = True)

has_vent_mode

##### Save the DataFrame with the ventilation modes as an excel files and the individual modes as text files

In [ ]:
for mode in ['sippv', 'simv', 'psv', 'mmv', 'hfov', 'vg', 'novg']:
    fhandle = open(os.path.join(DIR_WRITE, f'recording_list_{mode}.txt'), 'w')
    for recording in sorted(has_vent_mode[has_vent_mode[mode] == 1].index):
        fhandle.write( '%s ' % recording[0])
    fhandle.close()

In [ ]:
# Total number of recording parts
len(has_vent_mode)

In [ ]:
# Total number of patients
len(sorted(set(patient for patient, recording in has_vent_mode.index)))

In [ ]:
has_vent_mode.sum()

In [ ]:
has_vent_mode.to_excel(os.path.join(DIR_WRITE, 'vent_modes_all_draeger_all.xlsx'), sheet_name='vent_modes_all')

#### Consider only SIMV with or without VG

In [ ]:
has_simv_frme = has_vent_mode[has_vent_mode['simv'] == 1]
len(has_simv_frme)

In [ ]:
has_simv_frme.head()

In [ ]:
simv_only_frme = has_vent_mode[(has_vent_mode['simv'] == 1) & 
    (has_vent_mode[['cmv', 'sippv', 'simv', 'psv', 'mmv', 'hfov']].sum(axis=1) == 1)]
len(simv_only_frme)

In [ ]:
simv_only_frme.head()

In [ ]:
has_other_mode = set(has_simv_frme.index) -  set(simv_only_frme.index)
has_other_mode

In [ ]:
# Export vent mode frame as csv file and as pickle dump

has_simv_frme.to_csv(os.path.join(DIR_WRITE, 'has_simv_frme.csv'))

with open(os.path.join(DATA_DUMP, 'has_simv_frme.pickle'), 'wb') as handle:
    pickle.dump(has_simv_frme, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Patients who only received conventional ventilation
recordings_has_simv = list(has_simv_frme.index)
len(recordings_has_simv)

In [ ]:
recordings_has_simv[:3]

In [ ]:
# Patients who only received conventional ventilation
patients_has_simv = sorted(set(patient for patient, recording in recordings_has_simv))
len(patients_has_simv)

### 7. Limit clinical details to the selected recordings (only conventional modes with or without VG)

In [ ]:
clinical_details_recordings = \
    clinical_details_recordings[clinical_details_recordings['Patient'].isin(patients_has_simv)]
clinical_details_patients = \
    clinical_details_patients[clinical_details_patients['Patient'].isin(patients_has_simv)]

In [ ]:
clinical_details_recordings.head()

In [ ]:
clinical_details_patients.head()

In [ ]:
len(clinical_details_recordings), len(clinical_details_patients)

In [ ]:
# Export clinical data

clinical_details_recordings.to_csv(os.path.join(DIR_WRITE, 'clinical_details_recordings_simv.csv'))
clinical_details_patients.to_csv(os.path.join(DIR_WRITE, 'clinical_details_patients_simv.csv'))

with open(os.path.join(DATA_DUMP, 'clinical_details_recordings_simv.pickle'), 'wb') as handle:
    pickle.dump(clinical_details_recordings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(DATA_DUMP, 'clinical_details_patients_simv.pickle'), 'wb') as handle:
    pickle.dump(clinical_details_patients, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 8. Import ventilator parameters

In [ ]:
def column_selector(col):
    # There are sometimes extra columns in the imported DataFrames with no header. Exclude them.
    return '5001' in col or '8272' in col or col in ['Time [ms]', 'Date', 'Time', 'Rel.Time [s]',]

In [ ]:
def tag_remover(col_name):
    if '|' in col_name:
        return col_name.split('|')[1]
    else:
        return col_name

In [ ]:
def column_filter(dframe):
    to_choose_from = \
        ['MV [L/min]', 'MVi [L/min]', 'MVe [L/min]',  'MVemand [L/min]', 'MVespon [L/min]', 'MVleak [L/min]',
         'VT [mL]', 'VTmand [mL]', 'VTspon [mL]', 'VTimand [mL]', 'VTispon [mL]', 'VTemand [mL]', 'VTespon [mL]',   
         'PIP [mbar]', 'Pmean [mbar]', 'PEEP [mbar]', 
         'RR [1/min]', 'RRmand [1/min]', 'RRspon [1/min]', 'RRtrig [1/min]',
         'Tispon [s]', '% leak [%]', '% MVspon [%]', 'FiO2 [%]',]
    
    # Columns are not always present
    col_to_keep = sorted(set(dframe.columns) & set(to_choose_from))
    return(dframe[col_to_keep])

In [ ]:
def process_files_2(patient, recording, nrows = None):
  
    path = os.path.join(DIR_READ_VENT, patient, f'CsvLogBase_{recording}_slow_Measurement.csv')
    # This escaping characters with encoding errors with blackslashes 
    # that pd.csv can subsequently handle
    with open(path, encoding='utf8' , errors = 'backslashreplace',) as input_fd:
        dframe = pd.read_csv(input_fd, keep_date_col = 'True', usecols = column_selector, nrows = nrows,
        parse_dates = [['Date', 'Time']])
        
    dframe.index = dframe['Date_Time']
    dframe = dframe.rename(tag_remover, axis = 1)
    dframe = column_filter(dframe)
    # Resampling to remove half-empty rows
    # This resampling works because the 'mean()' methods ignores na values as a default
    return dframe.resample('1S').mean() 

In [ ]:
%%time

slow_measurements = defaultdict(dict)
for patient, recording in recordings_has_simv:
    print(datetime.now(), ' ', f'Working on {patient} - {recording}')
    slow_measurements[patient][recording] = process_files_2(patient, recording)  

### 9. Combine ventilator parameters and ventilator settings

In [ ]:
%%time

combined = defaultdict(dict)
for patient, recording in recordings_has_simv:
    #print(patient, recording)
    settings = vent_settings[patient][recording].reindex(slow_measurements[patient][recording].index, 
        method = 'ffill').copy()
    combined[patient][recording] = pd.merge(slow_measurements[patient][recording], settings,
        how = 'outer', left_index= True, right_index = True)

In [ ]:
combined.keys()

### 10. Correct relevant parameters to body weight

In [ ]:
clinical_details_recordings = clinical_details_recordings.set_index(['Patient', 'Recording'], drop=False)

In [ ]:
clinical_details_recordings.head()

In [ ]:
def weight_correct(patient, recording, dframe):
    
    to_weight_correct_potential = {'MV [L/min]', 'MVe [L/min]',
       'MVemand [L/min]', 'MVespon [L/min]', 'MVi [L/min]', 'MVleak [L/min]',
       'VTemand [mL]', 'VTespon [mL]', 'VTimand [mL]', 'VTispon [mL]', 'VTmand [mL]', 'VT [mL]',
       'VTspon [mL]', 'VT_set [mL]'}
    
    # Not all columns are always present
    to_weight_correct_actual = to_weight_correct_potential & set(dframe.columns)
    
    wt = clinical_details_recordings.loc[patient, recording]['Current weight']
   
    for par in to_weight_correct_actual:
        par_new = par[:-1] + '/kg' + par[-1]
        dframe[par_new] = dframe[par] / wt * 1000
        del dframe[par]
        
    return dframe

In [ ]:
%%time

for patient, recording in recordings_has_simv:
    print(patient, recording)
    combined[patient][recording] = weight_correct(patient, recording, combined[patient][recording])

### 11. Add Pinfl, VTdiff, Pdiff, RRdiff

In [ ]:
combined['LVD001']['2021-09-27_204445.625'].head()

In [ ]:
%%time

for patient, recording in recordings_has_simv:
    #print(patient, recording)
    
    if len({'PIP [mbar]', 'PEEP_set [mbar]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['Pinfl_mand [mbar]'] = combined[patient][recording]['PIP [mbar]'] - \
        combined[patient][recording]['PEEP_set [mbar]']
        
    if len({'PS_set [mbar]', 'PEEP_set [mbar]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['PIP_spon [mbar]'] = combined[patient][recording]['PS_set [mbar]'] + \
        combined[patient][recording]['PEEP_set [mbar]']
    
    if len({'PIP [mbar]', 'Pmax_set [mbar]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['Pdiff [mbar]'] = combined[patient][recording]['Pmax_set [mbar]'] - \
        combined[patient][recording]['PIP [mbar]']
            
    # In some recordings there is only VT rather than VTmand. However, prefer VTmand
    if len({'VT [mL/kg]', 'VT_set [mL/kg]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['VTdiff [mL/kg]'] = combined[patient][recording]['VT [mL/kg]'] - \
        combined[patient][recording]['VT_set [mL/kg]']
    
    if len({'VTmand [mL/kg]', 'VT_set [mL/kg]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['VTdiff [mL/kg]'] = combined[patient][recording]['VTmand [mL/kg]'] - \
        combined[patient][recording]['VT_set [mL/kg]']
    
    if len({'RRmand [1/min]', 'RR_set [1/min]'} & set(combined[patient][recording].columns)) == 2:
        combined[patient][recording]['RRdiff [1/min]'] = \
        combined[patient][recording]['RRmand [1/min]'] - combined[patient][recording]['RR_set [1/min]']
        
    # sort columns
    combined[patient][recording].sort_index(axis=1, inplace = True)

### 12. Add info about leak compensation

This information is not included directly in the downloaded data. However, when leak compensation is off, the VTmand is essentially (within 0.001 mL/kg) identical to VTemand. When leak compensation is on, VTmand > VTemand because there is always some, even if minimal, leak

In the recordings where VTmand is not available use VT

In [ ]:
for patient, recording in recordings_has_simv:
    if 'VTmand [mL/kg]' in combined[patient][recording].columns and \
        'VTemand [mL/kg]' in combined[patient][recording].columns:
        print(patient, recording, 'VTmand-VTemand = ', (combined[patient][recording]['VTmand [mL/kg]'] - 
            combined[patient][recording]['VTemand [mL/kg]']).mean())
        
    elif 'VT [mL/kg]' in combined[patient][recording].columns and \
        'VTemand [mL/kg]' in combined[patient][recording].columns:
        print(patient, recording, 'VT-VTemand = ', (combined[patient][recording]['VT [mL/kg]'] - 
            combined[patient][recording]['VTemand [mL/kg]']).mean())

In [ ]:
for patient, recording in recordings_has_simv:
    if 'VTmand [mL/kg]' in combined[patient][recording].columns and \
        'VTemand [mL/kg]' in combined[patient][recording].columns:
        if (combined[patient][recording]['VTmand [mL/kg]'] - 
            combined[patient][recording]['VTemand [mL/kg]']).mean() < 0.001:
            combined[patient][recording]['leak compensation'] = 'off'
        else:
            combined[patient][recording]['leak compensation'] = 'on'
    
    elif 'VT [mL/kg]' in combined[patient][recording].columns and \
         'VTemand [mL/kg]' in combined[patient][recording].columns:
        if (combined[patient][recording]['VT [mL/kg]'] - 
            combined[patient][recording]['VTemand [mL/kg]']).mean() < 0.001:
            combined[patient][recording]['leak compensation'] = 'off'
        else:
            combined[patient][recording]['leak compensation'] = 'on'

In [ ]:
# Make the leak compensation categorical
for patient, recording in recordings_has_simv:
    combined[patient][recording]['leak compensation'] = \
        combined[patient][recording]['leak compensation'].astype('category')

In [ ]:
combined[patient][recording].head()

### 13. For those recordings which have other modes, keep only SIMV

In [ ]:
has_other_mode

In [ ]:
vent_modes['LVD002']['2021-11-08_105631.989'];

In [ ]:
combined['LVD002']['2021-11-08_105631.989'] = \
    combined['LVD002']['2021-11-08_105631.989'].loc[:'2021-11-09 13:10:24']

In [ ]:
vent_modes['LVD004']['2021-10-25_211444.182'];

In [ ]:
combined['LVD004']['2021-10-25_211444.182'] = \
    combined['LVD004']['2021-10-25_211444.182'].loc[:'2021-10-26 11:37:15']

In [ ]:
vent_modes['LVD005']['2021-12-16_092336.151'];

In [ ]:
combined['LVD005']['2021-12-16_092336.151'] = \
    combined['LVD005']['2021-12-16_092336.151'].loc[:'2021-12-20 22:40:08']

In [ ]:
vent_modes['LVD005']['2021-12-22_203909.404'];

In [ ]:
combined['LVD005']['2021-12-22_203909.404'] = \
    combined['LVD005']['2021-12-22_203909.404'].loc[:'2021-12-26 12:20:20']

In [ ]:
vent_modes['LVD005']['2021-12-28_154455.897'];

In [ ]:
combined['LVD005']['2021-12-28_154455.897'] = \
    combined['LVD005']['2021-12-28_154455.897'].loc[:'2021-12-31 14:13:09']

In [ ]:
vent_modes['LVD008']['2021-12-04_132746.283'];

In [ ]:
combined['LVD008']['2021-12-04_132746.283'] = \
    combined['LVD008']['2021-12-04_132746.283'].loc['2021-12-04 15:24:14':]

In [ ]:
vent_modes['LVD008']['2021-12-06_102618.106'];

In [ ]:
combined['LVD008']['2021-12-04_132746.283'] = \
    combined['LVD008']['2021-12-04_132746.283'].loc[:'2021-12-07 16:57:35']

In [ ]:
vent_modes['LVD009']['2021-12-09_123914.130'];

In [ ]:
combined['LVD009']['2021-12-09_123914.130'] = \
    combined['LVD009']['2021-12-09_123914.130'].loc[:'2021-12-11 20:07:32']

In [ ]:
vent_modes['LVD010']['2021-12-09_122846.010'];

In [ ]:
combined['LVD010']['2021-12-09_122846.010'] = \
    combined['LVD010']['2021-12-09_122846.010'].loc[:'2021-12-10 08:06:45']

In [ ]:
vent_modes['LVD013']['2022-01-07_151845.678'];

In [ ]:
combined['LVD013']['2022-01-07_151845.678'] = \
    combined['LVD013']['2022-01-07_151845.678'].loc[:'2022-01-11 08:46:13']

In [ ]:
vent_modes['LVD013']['2022-02-11_095849.673'];

In [ ]:
combined['LVD013']['2022-02-11_095849.673'] = \
    combined['LVD013']['2022-02-11_095849.673'].loc[:'2022-02-12 22:14:54']

In [ ]:
vent_modes['LVD015']['2022-01-22_194526.833'];

In [ ]:
combined['LVD015']['2022-01-22_194526.833'] = \
    combined['LVD015']['2022-01-22_194526.833'].loc[:'2022-01-23 19:52:13']

In [ ]:
vent_modes['LVD016']['2022-01-21_113956.352'];

In [ ]:
combined['LVD016']['2022-01-21_113956.352'] = \
    combined['LVD016']['2022-01-21_113956.352'].loc['2022-01-21 11:45:04':]

### 14. Export data

#### Ventilator parameters and settings

In [ ]:
%%time
for patient in sorted(combined.keys()):
    for recording in combined[patient]:
        print(f'Exporting {patient}  {recording}')
        combined[patient][recording].to_hdf(os.path.join(DATA_DUMP, f'{patient}_{recording}_pars_and_settings.h5'),
        format = 'table', key = patient)